Contexto

---

Exercício prático do Capítulo 3 de \cite{IvanNunes_RNA:2010} trata de um problema no processo de destilação fracionada de petróleo referente à classificação de um determinado óleo em duas classes: $P_1$ e $P_2$, a partir da análise de três grandezas referentes à propriedades físico-químicas: $x_1$, $x_2$ e $x_3$.

Para realizar essa classificação a equipe de engenheira da empresa de destilação fracionada optou pela rede perceptron com três entradas e limiar de ativação $\theta = -1$, o qual representa a classe $P_1$ e 1 representa a $P_2$. A arquitetura pode ser visualizada na Figura a seguir.

<img src="Fig_Ex_3.6.png">

Bibliotecas utilizadas

---

In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

Tratamento dos dados

---

In [32]:
# Leitura dos dados

data = pd.read_csv('data_36_RNA.csv')

In [33]:
data

,x1,x2,x3,d
0,-0.6508,0.1097,4.0009,-1
1,-1.4492,0.8896,4.4005,-1
2,2.0850,0.6876,12.0710,-1
3,0.2626,1.1476,7.7985,1
4,0.6418,1.0234,7.0427,1
5,0.2569,0.6730,8.3265,-1
6,1.1155,0.6043,7.4446,1
7,0.0914,0.3399,7.0677,-1
8,0.0121,0.5256,4.6316,1
9,-0.0429,0.4660,5.4323,1


In [34]:
# Separacao dos dados

x1 = data['x1'].copy()
x2 = data['x2'].copy()
x3 = data['x3'].copy()
X = pd.concat([x1, x2, x3], axis=1)

y = data['d'].copy()

In [53]:
# Normalizacao dos dados segundo escala minmax

def normalizacao(data):

    min = data.values.min()
    max = data.values.max()

    data_norm = data.copy()

    for j in data.columns:

        for i in data.index:

            x = data.loc[i, j]

            x_norm = x / max

            data_norm.loc[i, j] = x_norm

    return data_norm

In [54]:
X_norm = normalizacao(X)

In [55]:
# Adiciona coluna do limiar de ativacao = -1 (bias) no conjunto de dados
 
X_norm['bias'] = -1

In [56]:
X_norm = np.array(X_norm)


In [57]:
X_norm

array([[-0.05391434,  0.0090879 ,  0.33144727, -1.        ],
       [-0.12005633,  0.07369729,  0.3645514 , -1.        ],
       [ 0.17272803,  0.05696297,  1.        , -1.        ],
       [ 0.02175462,  0.09507083,  0.64605252, -1.        ],
       [ 0.05316875,  0.08478171,  0.58343965, -1.        ],
       [ 0.02128241,  0.05575346,  0.68979372, -1.        ],
       [ 0.09241156,  0.05006213,  0.61673432, -1.        ],
       [ 0.00757187,  0.0281584 ,  0.58551073, -1.        ],
       [ 0.0010024 ,  0.04354237,  0.38369646, -1.        ],
       [-0.00355397,  0.03860492,  0.450029  , -1.        ],
       [ 0.03595394,  0.05691326,  0.68169166, -1.        ],
       [ 0.02265761,  0.08522078,  0.59592412, -1.        ],
       [ 0.04008781,  0.04018723,  0.62008119, -1.        ],
       [ 0.03387458, -0.01049623,  0.45579488, -1.        ],
       [ 0.11921962,  0.01337089,  0.71115069, -1.        ],
       [-0.07551156, -0.01634496,  0.18194019, -1.        ],
       [ 0.0302709 ,  0.

Treinamento da rede neural

---

In [58]:
# valor maximo de epocas
MAX_EPOCA = 10000

# Taxa de aprendizado
n = 0.01

# Pesos iniciais (wi) e ajustados/finais (wf)
wi = pd.DataFrame(columns=['w0', 'w1', 'w2', 'w3'])
wf = pd.DataFrame(columns=['w0', 'w1', 'w2', 'w3'])

In [16]:
def func_ativacao(u):

    n = len(u)

    # Funcao Degrau Bipolar (-1;1)
    g = np.full(n, 1)

    # Verificacao: se u < 0, y = -1
    for i in range(n):

        if u[i] < 0:
            
            g[i] = -1

    return g

In [29]:
def treinamento(n, X_norm, wi, wf, MAX_EPOCA, y):

    y = np.array(y)

    for n_treino in range(5):

        print('-'*30)
        print(f'Treinamento numero: {n_treino}')
        print('-'*30)
        
        # Pesos aleatorios entre 0 e 1 do treinamento
        w = np.random.rand(4, 1)
        
        print(f'Pesos iniciais aleatorios: \n {w}')

        # Salva na tabela wi
        wi.loc[n_treino, :] = w[:, 0]

        # Contador de epocas
        epoca = 0

        print('-'*30)
        while(epoca < MAX_EPOCA):

            # Multiplicacao dos dados de entrada e bias (X_norm) pelos pesos (w)
            u = X_norm.dot(w)

            # Matriz resultante da multiplicacao das entradas pelos pesos (xn*wn) em formato de vetor
            u = np.array(u)

            # Verificacao do sinal de saida pela funcao de ativacao
            y_calc = func_ativacao(u)

            # Se os resultados calculados sao os mesmo que a saida
            if np.all(y_calc == y):

                wf.loc[n_treino, :] = w[:, 0]

                epoca = epoca + 1

                print(f'Convergência em {epoca} épocas')
                print(f'Pesos finais: \n {w}')

                break
            
            # Regra de Hebb
            else:

                erro = y - y_calc

                x_T = X_norm.T

                x_T_erro = x_T.dot(erro)
                
                n_x_T_erro = n*x_T_erro

                n_x_T_erro = n_x_T_erro.reshape(-1, 1)
                
                w = w + n_x_T_erro

                epoca = epoca + 1

    return wi, wf

Teste da rede neural

---

In [62]:
data_teste = pd.read_csv('data_teste_36_RNA.csv')

In [63]:
data_teste

,x1,x2,x3
0,-0.3665,0.0620,5.9891
1,-0.7842,1.1267,5.5912
2,0.3012,0.5611,5.8234
3,0.7757,1.0648,8.0677
4,0.1570,0.8028,6.3040
5,-0.7014,1.0316,3.6005
6,0.3748,0.1536,6.1537
7,-0.6920,0.9404,4.4058
8,-1.3970,0.7141,4.9263
9,-1.8842,-0.2805,1.2548


In [64]:
X_teste_norm = normalizacao(data_teste)

In [65]:
X_teste_norm['bias'] = -1

In [66]:
X_teste_norm = np.array(X_teste_norm)

In [67]:
X_teste_norm

array([[-0.04542806,  0.00768497,  0.74235532, -1.        ],
       [-0.09720242,  0.13965566,  0.69303519, -1.        ],
       [ 0.03733406,  0.06954894,  0.72181663, -1.        ],
       [ 0.09614884,  0.13198309,  1.        , -1.        ],
       [ 0.01946032,  0.09950791,  0.78138751, -1.        ],
       [-0.08693928,  0.12786792,  0.44628581, -1.        ],
       [ 0.04645686,  0.01903888,  0.76275766, -1.        ],
       [-0.08577414,  0.11656358,  0.5461036 , -1.        ],
       [-0.17315964,  0.08851345,  0.61062013, -1.        ],
       [-0.2335486 , -0.03476827,  0.1555338 , -1.        ]])

In [25]:
def teste(X, wf):

    wf_T = wf.transpose()
    wf_T = np.array(wf_T)

    u_teste = X.dot(wf_T)

    for j in range(5):

        print('-'*30)
        print(f'Teste numero: {j}')
        print('-'*30)

        y_teste = func_ativacao(u_teste[:,j])

        print(y_teste)

    return u_teste

Aplicacação da rede neural

---

In [27]:
def app(n, X_norm, wi, wf, MAX_EPOCA, y, X_teste_norm):

    pesos_inicias, pesos_finais = treinamento(n, X_norm, wi, wf, MAX_EPOCA, y)

    resultado = teste(X_teste_norm, pesos_finais)

    return pesos_inicias, pesos_finais, resultado

In [68]:
pesos_inicias, pesos_finais, resultado = app(n, X_norm, wi, wf, MAX_EPOCA, y, X_teste_norm)

------------------------------
Treinamento numero: 0
------------------------------
Pesos iniciais aleatorios: 
 [[0.9629826 ]
 [0.82820938]
 [0.48389599]
 [0.0346685 ]]
------------------------------
Convergência em 1214 épocas
Pesos finais: 
 [[ 2.99847544]
 [ 4.36855417]
 [-1.39472583]
 [-0.5053315 ]]
------------------------------
Treinamento numero: 1
------------------------------
Pesos iniciais aleatorios: 
 [[0.28705101]
 [0.12137788]
 [0.54127801]
 [0.43244665]]
------------------------------
Convergência em 1281 épocas
Pesos finais: 
 [[ 2.86860515]
 [ 4.58385655]
 [-1.37963062]
 [-0.50755335]]
------------------------------
Treinamento numero: 2
------------------------------
Pesos iniciais aleatorios: 
 [[0.09853149]
 [0.20614558]
 [0.59826684]
 [0.15991807]]
------------------------------
Convergência em 1312 épocas
Pesos finais: 
 [[ 2.81341658]
 [ 4.72795653]
 [-1.37870226]
 [-0.48008193]]
------------------------------
Treinamento numero: 3
-----------------------------

Resultados

---

In [70]:
pesos_inicias

,w0,w1,w2,w3
0,0.962983,0.828209,0.483896,0.034668
1,0.287051,0.121378,0.541278,0.432447
2,0.098531,0.206146,0.598267,0.159918
3,0.374544,0.949356,0.515427,0.260788
4,0.687295,0.013927,0.533471,0.106669


In [71]:
pesos_finais

,w0,w1,w2,w3
0,2.998475,4.368554,-1.394726,-0.505332
1,2.868605,4.583857,-1.379631,-0.507553
2,2.813417,4.727957,-1.378702,-0.480082
3,2.545206,4.45348,-1.268643,-0.439212
4,2.97479,4.321679,-1.372374,-0.493331
